# Human Feedback in the Loop

In [ ]:
## Create LLM
from langchain_ollama import ChatOllama

llm = ChatOllama(model="llama3.2:1b", temperature=0)

In [ ]:
## Create memory for LLM
from langgraph.checkpoint.memory import MemorySaver

memory=MemorySaver()

In [ ]:
## Create State
from typing import Annotated
from typing_extensions import TypedDict
from langgraph.graph.message import add_messages

class State(TypedDict):
    messages:Annotated[list, add_messages]

In [ ]:
from langchain_core.tools import tool
from langgraph.graph import StateGraph, START, END
from langgraph.prebuilt import ToolNode
from langgraph.prebuilt import tools_condition

from langgraph.types import Command, interrupt

## Create graph builder
graph_builder=StateGraph(State)

In [ ]:
## Human Assistance Tool
@tool
def human_assistance(query:str) -> str:
    """Use this tool if assistance is requested from a user."""
    human_response = interrupt({ "query": query })
    return human_response["data"]


## Multiply Tool
@tool
def multiply(a:int,b:int) -> int:
    """
    Multiple two numbers
    
    a: first int
    b: second int

    Retruns:
        int result of multiplication
    """
    return a*b

tools = [multiply, human_assistance]
llm_with_tools = llm.bind_tools(tools)

In [ ]:
## Node definition
def tool_calling_llm(state:State):
    return {"messages":[llm_with_tools.invoke(state["messages"])]}

## Graph
builder=StateGraph(State)
builder.add_node("tool_calling_llm", tool_calling_llm)
builder.add_node("tools",ToolNode(tools))

## Add Edges
builder.add_conditional_edges("tool_calling_llm", tools_condition)

builder.add_edge("tools", "tool_calling_llm")
builder.add_edge(START, "tool_calling_llm")

graph = builder.compile(checkpointer=memory)
graph

In [ ]:
config={"configurable":{"thread_id":"1"}}

In [ ]:
user_input = "I need some expert guidance and assistance for building my AI Agent. could you request assistance for me?"

events = graph.stream({"messages":user_input},
                      config,
                      stream_mode="values")
for event in events:
    if "messages" in event:
        event["messages"][-1].pretty_print()

In [ ]:
human_response = (
    "We, the experts are here to help! We'd recommend you check out LangGraph to build your agent."
    "It's much more reliable and extensible than simple automation agents."
)

human_command = Command(resume={"data": human_response})

events = graph.stream(human_command, 
                      config, 
                      stream_mode="values")

for event in events:
    if "messages" in event:
        event["messages"][-1].pretty_print()